In [1]:
import pandas as pd
import numpy as np
import math

#### Text Preprocessing

In [2]:
from utils import preprocessText

In [3]:
text = "And one only, in my opinion.<br /><br />That reason is Margaret Leighton. She is a wonderful actress, on-stage as well as on the screen. We have few chances to see her, though. I think that\'s especially true in the United States.<br /><br />Here she plays a sympathetic role. Not only that but she is also very pretty and meant to be something of a bombshell.<br /><br />Walter Pigeon does not hold up the tradition of Drummond performers. He is always reliable but he's not much fun. He's not a rascal or a knave. Consequently, this seemed to me a talky endeavor with little action or suspense. But check it out for Leighton."

In [4]:
# combineText(splitter.split(text))
preprocessText(text)

"And one only, in my opinion. That reason is Margaret Leighton. She is a wonderful actress, onstage as well as on the screen. We have few chances to see her, though. I think that's especially true in the United States. Here she plays a sympathetic role. Not only that but she is also very pretty and meant to be something of a bombshell. Walter Pigeon does not hold up the tradition of Drummond performers. He is always reliable but he's not much fun. He's not a rascal or a knave. Consequently, this seemed to me a talky endeavor with little action or suspense. But check it out for Leighton."

#### Preparing Mutant Template

Please refer to `codes/mutant-generation.ipynb`to know the detail in getting the template.

In [5]:
dfm = pd.read_csv("../data/imdb_mutant/male/test.csv", header=None, sep="\t", names=["label", "mutant", "template"])
dff = pd.read_csv("../data/imdb_mutant/female/test.csv", header=None, sep="\t", names=["label", "mutant", "template"])

In [6]:
dfm

,label,mutant,template
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
...,...,...,...
138995,1,"First, I'm a huge Justin fan. I grew up knowin...","First, I'm a huge Buddy Holly fan. I grew up k..."
138996,1,"First, I'm a huge Terrence fan. I grew up know...","First, I'm a huge Buddy Holly fan. I grew up k..."
138997,1,"First, I'm a huge Roger fan. I grew up knowing...","First, I'm a huge Buddy Holly fan. I grew up k..."
138998,1,"First, I'm a huge Torrance fan. I grew up know...","First, I'm a huge Buddy Holly fan. I grew up k..."


In [7]:
df = pd.concat([dfm, dff])

In [8]:
df

,label,mutant,template
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
...,...,...,...
138995,1,"First, I'm a huge Melanie fan. I grew up knowi...","First, I'm a huge Buddy Holly fan. I grew up k..."
138996,1,"First, I'm a huge Tanisha fan. I grew up knowi...","First, I'm a huge Buddy Holly fan. I grew up k..."
138997,1,"First, I'm a huge Nancy fan. I grew up knowing...","First, I'm a huge Buddy Holly fan. I grew up k..."
138998,1,"First, I'm a huge Tia fan. I grew up knowing w...","First, I'm a huge Buddy Holly fan. I grew up k..."


In [9]:
df["template"] = df["template"].astype("category")
df["template_id"] = df["template"].cat.codes

In [10]:
gb = df.groupby("template_id")

In [11]:
gb.count()

,label,mutant,template
template_id,,,
0,40,40,40
1,40,40,40
2,40,40,40
3,40,40,40
4,40,40,40
...,...,...,...
6893,40,40,40
6894,40,40,40
6895,40,40,40


we have 6898 templates with 40 mutants for each template

#### Mutant Generation

In [1]:
import spacy
import pandas as pd

from utils import preprocessText

In [2]:
from Entity import Entity
from CustomToken import CustomToken as Token
from Coreference import Coreference
from MutantGeneration import MutantGeneration

In [9]:
# the main reference is a person name
# text = "When Nathaniel Kahn embarked into this voyage, he hardly knew who his father really was. By the end of the film, he found him and comes to terms with the strange life he lived as a child. Louis Kahn was the father."

# the person name is a substring of the main reference
# text = "Meek and mild Edward G. Robinson (as Wilbert Winkle) decides to quit his bank job and do what he wants, open a ”fix-it” repair shop behind his house. Mr. Robinson is married, but childless; he has befriended local orphanage resident Ted Donaldson (as Barry)"

# the main reference is a gender associated word
text = "Even the manic loony who hangs out with the bad guys in ”Mad Max” is there. That guy from ”Blade Runner” also cops a good billing, although he only turns up at the beginning and the end of the movie."

# the main reference is a pronoun
# text = "This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway. See! Cody Powers Jarrett blow himself up on his biggest gas container yet!"
# text = "This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway. SEE! CODY POWERS JARRETT BLOW HIMSELF UP ON HIS BIGGEST GAS CONTAINER YET! TONIGHT! 7.30PM!  However, one day, his mommy dies and Jarrett goes berserk. He kidnaps the audience in the theatre and makes them all stand on top of a huge gas cylinder. Losing control further, he makes them all scream \"MADE IT MA, TOP OF THE CYLINDER!\" in unison. The noise is so deafening that it bursts Jarrets eardrums, causing him to topple from the cylinder into a vat of acid. This Warner Bros. movie is not all it's cracked up to be."


# random
# text = "About your terrible movie copying Beethoven. As a professional musician it\'s my duty to watch every movie made about any composer and Beethoven is one of my favorites. When Hungarians and Americans meet, it\'s a terrible combination of empty over the top emotions combined with the worst taste possible. You proved it in your terrible b-movie. The only thing that carries the movie is the music. Of course you didn\'t bother to look further than the good but in my taste contrived performances of the Tackacs quartet, but OK I have to admit that the performances at least have quality as contrast to the movie you\'ve made. It starts of with the dying DEAF Beethoven who perfectly understands Anna who is merely whispering. Beethoven\'s hearing during the movie get\'s better by the minute, but that must be because of some vague divine thing. Then there is the quite impossible semi-pornographic \"eyes wide shut\" double-conducting scene which is totally over the top with the luscious Anna and the crying nephew in the end (who also cries in the deleted scenes with constant red eyes, my GOD what a performance). And as culmination the rip-off from Amadeus, with Beethoven dictating music to Anna not in notes but in total nonsense, which she understands perfectly but no-one else in your audience even trained professional musicians will understand. Of course your reaction will be that negative response is a response at least, but I can assure you that Beethoven himself is turning in his grave because of your worthless creation and with reason. This so called homage is blasphemy and I am so sorry to have rented one of the worst movies ever made even though it\'s about my favorite subject. Ed Harris and others, you cannot comprehend the greatness of Beethoven in your wildest dreams and certainly not after a couple of lessons in conducting and violin playing. That's the trouble with you Americans: you think you can grasp everything even when it takes a lifetime of hard work. Yeah we can do it anyway! Remember that a good product comes with hard labor, talent, devotion and professionalism. All these you creators of Copying Beethoven lack. See you in kindergarten."

In [10]:
text = preprocessText(text)
print(text)
m = MutantGeneration(text)

Even the manic loony who hangs out with the bad guys in ” Mad Max ” is there. That guy from ” Blade Runner ” also cops a good billing, although he only turns up at the beginning and the end of the movie.
['That guy from ” Blade Runner', 'he']


In [11]:
m.generateTemplate()

The Main Reference is a Gender Associated Word
That <gaw> from ” Blade Runner
Even the manic loony who hangs out with the bad guys in ” Mad Max ” is there.  That <gaw> from ” Blade Runner  ” also cops a good billing, although  <pro-he>  only turns up at the beginning and the end of the movie.


In [ ]:
m.getResolved()

In [7]:
for r in m.getCoreferences() :
    print(r.getMainReference())
    print(r.getReferences())

That guy from ”Blade Runner
['That guy from ”Blade Runner', 'he']


In [8]:
m.isHavingOnePersonReference()

True

In [9]:
if m.isHavingOnePersonReference() :
    print(m.getPersonReference())
#     print(m.generate_mutant_text())

That guy from ”Blade Runner: ['That guy from ”Blade Runner', 'he']


#### Load Name from Gender Computer

In [3]:
gc = pd.read_csv("../data/gc_name/data.csv")
gc

,Name,Gender,Country
0,Roen,male,UK
1,Jeet,male,UK
2,Hagen,male,UK
3,Willow,male,UK
4,Belal,male,UK
...,...,...,...
305,Dódi,female,Hungary
306,Iboyka,female,Hungary
307,Zsófia,female,Hungary
308,Ágota,female,Hungary


In [4]:
gcm = gc[gc["Gender"] == "male"]
gcf = gc[gc["Gender"] == "female"]

In [5]:
# names from GC
mnames = gcm["Name"].values
fnames = gcf["Name"].values

# names from EEC paper
# mnames = ["Alonzo", "Adam", "Alphonse", "Alan", "Darnell", "Andrew", "Jamel", "Frank", "Jerome", "Harry", "Lamar", "Jack", "Leroy", "Josh", "Malik", "Justin", "Terrence", "Roger", "Torrance", "Ryan"]
# fnames = ["Ebony", "Amanda", "Jasmine", "Betsy", "Lakisha", "Courtney", "Latisha", "Ellen", "Latoya", "Heather", "Nichelle", "Katie", "Shaniqua", "Kristin", "Shereen", "Melanie", "Tanisha", "Nancy", "Tia", "Stephanie"]

# small name for debugging
mnames = ["Alonzo"] 
fnames = ["Ebony"]

In [65]:
dt = df[["label", "template"]]

In [66]:
dt = dt.drop_duplicates().reset_index(drop=True)
dt

,label,template
0,1,"I have only see three episodes of Hack, starri..."
1,1,In the groovy mid 70's a scruffy bunch of bras...
2,1,This must have been one of Chaplin's most ambi...
3,1,The debut that plucked from obscurity one of t...
4,1,There is really no way to compare this motion ...
...,...,...
6893,1,I heard they were going to remake this French ...
6894,1,"Well, the movie did turn out a lot better than..."
6895,0,"In this film, there is a loose plot of a man (..."
6896,1,The French Babbette appears at the modest hous...


In [69]:
import time
start = time.time()

original_arr = []
mutant_arr = []
gender_arr = []
label_arr = []
generation_approach_arr = []
person_reference_arr = []

for index, row in dt.iterrows():
    label = row['label']
    text = row['template']
    c = Coref(text)
    if c.is_having_one_subject() :

        # append original text
        original_arr.append(text)
        mutant_arr.append(text)
        gender_arr.append("template")
        person_reference_arr.append("template")
        generation_approach_arr.append("template")
        label_arr.append(label)
        
        mtext = c.generate_mutants()
        for m in mtext :
            original_arr.append(text)
            mutant_arr.append(m["text"])
            gender_arr.append(m["gender"])
            person_reference_arr.append(m["person_reference"])
            generation_approach_arr.append(m["mutant_generation"])
            label_arr.append(label)
        

end = time.time()
print("Execution Time: ", end-start)

Execution Time:  1041.3692698478699


In [70]:
dmutant = pd.DataFrame(data={"label": label_arr, "mutant": mutant_arr, "gender": gender_arr, "original": original_arr, "person_reference": person_reference_arr, "generation_approach": generation_approach_arr})
dmutant

,label,mutant,gender,original,person_reference,generation_approach
0,1,"I have only see three episodes of Hack, starri...",template,"I have only see three episodes of Hack, starri...",template,template
1,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Alonzo,normal
2,1,"I have only see three episodes of Hack, starri...",female,"I have only see three episodes of Hack, starri...",Ebony,normal
3,1,In the groovy mid 70's a scruffy bunch of bras...,template,In the groovy mid 70's a scruffy bunch of bras...,template,template
4,1,In the groovy mid 70's a scruffy bunch of bras...,male,In the groovy mid 70's a scruffy bunch of bras...,Alonzo,normal
...,...,...,...,...,...,...
29067,1,"Well, the movie did turn out a lot better than...",template,"Well, the movie did turn out a lot better than...",template,template
29068,1,"Well, the movie did turn out a lot better than...",male,"Well, the movie did turn out a lot better than...",Alonzo,normal
29069,1,"Well, the movie did turn out a lot better than...",female,"Well, the movie did turn out a lot better than...",Ebony,normal
29070,1,The French Babbette appears at the modest hous...,template,The French Babbette appears at the modest hous...,template,template


In [71]:
dmutant["original"] = dmutant["original"].astype("category")
dmutant["template_id"] = dmutant["original"].cat.codes

In [72]:
dmutant.groupby("template_id").count()

,label,mutant,gender,original,person_reference,generation_approach
template_id,,,,,,
0,3,3,3,3,3,3
1,3,3,3,3,3,3
2,1,1,1,1,1,1
3,27,27,27,27,27,27
4,3,3,3,3,3,3
...,...,...,...,...,...,...
6077,27,27,27,27,27,27
6078,1,1,1,1,1,1
6079,1,1,1,1,1,1


In [73]:
dms = dmutant[dmutant["gender"] == "template"]
dmd = dmutant[dmutant["gender"] != "template"]

In [74]:
dmd = dmd.reset_index(drop=True)
dmd

,label,mutant,gender,original,person_reference,generation_approach,template_id
0,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Alonzo,normal,1845
1,1,"I have only see three episodes of Hack, starri...",female,"I have only see three episodes of Hack, starri...",Ebony,normal,1845
2,1,In the groovy mid 70's a scruffy bunch of bras...,male,In the groovy mid 70's a scruffy bunch of bras...,Alonzo,normal,2830
3,1,In the groovy mid 70's a scruffy bunch of bras...,female,In the groovy mid 70's a scruffy bunch of bras...,Ebony,normal,2830
4,1,This must have been one of Alonzo's most ambit...,male,This must have been one of Chaplin's most ambi...,Alonzo,normal,5199
...,...,...,...,...,...,...,...
22985,0,My Take: Yet another lame PG-13 horror movie w...,female,My Take: Yet another lame PG-13 horror movie w...,Ebony,normal,3325
22986,1,I heard they were going to remake this French ...,male,I heard they were going to remake this French ...,Alonzo ( Jean Servais,part of person reference,1898
22987,1,I heard they were going to remake this French ...,female,I heard they were going to remake this French ...,Ebony ( Jean Servais,part of person reference,1898
22988,1,"Well, the movie did turn out a lot better than...",male,"Well, the movie did turn out a lot better than...",Alonzo,normal,5541


In [47]:
dmd.groupby("template_id").count()

,label,mutant,gender,original,name
template_id,,,,,
0,310,310,310,310,310
1,310,310,310,310,310
2,310,310,310,310,310
3,310,310,310,310,310
4,310,310,310,310,310
...,...,...,...,...,...
6078,310,310,310,310,310
6079,310,310,310,310,310
6080,310,310,310,310,310


In [48]:
dms = dms.reset_index(drop=True)
dms

,label,mutant,gender,original,name,template_id
0,1,"I have only see three episodes of Hack, starri...",template,"I have only see three episodes of Hack, starri...",template,1846
1,1,In the groovy mid 70's a scruffy bunch of bras...,template,In the groovy mid 70's a scruffy bunch of bras...,template,2831
2,1,This must have been one of Chaplin's most ambi...,template,This must have been one of Chaplin's most ambi...,template,5200
3,1,The debut that plucked from obscurity one of t...,template,The debut that plucked from obscurity one of t...,template,4244
4,1,There is really no way to compare this motion ...,template,There is really no way to compare this motion ...,template,4491
...,...,...,...,...,...,...
6078,0,My Take: Yet another lame PG-13 horror movie w...,template,My Take: Yet another lame PG-13 horror movie w...,template,3326
6079,1,I heard they were going to remake this French ...,template,I heard they were going to remake this French ...,template,1899
6080,1,"Well, the movie did turn out a lot better than...",template,"Well, the movie did turn out a lot better than...",template,5542
6081,1,The French Babbette appears at the modest hous...,template,The French Babbette appears at the modest hous...,template,4152


In [49]:
c = dms.groupby("template_id").count()
c

,label,mutant,gender,original,name
template_id,,,,,
0,1,1,1,1,1
1,1,1,1,1,1
2,1,1,1,1,1
3,1,1,1,1,1
4,1,1,1,1,1
...,...,...,...,...,...
6078,1,1,1,1,1
6079,1,1,1,1,1
6080,1,1,1,1,1


In [50]:
# dms.groupby("template_id").get_group(302)["mutant"].values[1]

In [51]:
import os

dirname = "../data/gc_imdb/"

if not os.path.exists(dirname) :
    os.makedirs(dirname)

dmutant.to_csv(dirname + "test.csv", index=None, header=None, sep="\t")